In [1]:
import pandas as pd
import load_data

In [2]:
questions, context = load_data.get_siqa_data()

Context: Tracy didn't go home that evening and resisted Riley's attacks.
Question: What does Tracy need to do before this?
Correct Label: 3


In [3]:
context_dict = {'context': context[:1000]}  
context_df = pd.DataFrame(context_dict) 
    
# write the DataFrame to a CSV file
# df.to_csv('SIQA_context_1000.csv', index=True, index_label='ID') 

In [4]:
# empty gpu
import gc
import torch

torch.cuda.empty_cache() 
gc.collect()

0

In [5]:
# !wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q6_K.gguf

In [6]:
!gpustat -cp

torch-cuda-568fb8d4d6-kqdjc  Sun Nov  5 20:09:29 2023  515.65.01
[0] Tesla P40 | 30°C,   0 % |   120 / 23040 MB |


In [7]:
import llama_cpp
from llama_cpp import Llama

In [8]:
# model_path = "BIG-bench/llama-2-13b.Q4_0.gguf"
model_path = "BIG-bench/llama-2-13b-chat.Q6_K.gguf"

In [9]:
llm = Llama(model_path=model_path, n_gpu_layers=-1, n_ctx=4096, logits_all=True, verbose=False)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla P40, compute capability 6.1
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from BIG-bench/llama-2-13b-chat.Q6_K.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q6_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q6_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q6_K     [  5120,  5120,     1,     1 ]
llama_model_

In [14]:
def get_llm_output(prompt):
  response = ""
  response = llm(prompt, temperature=0.4, repeat_penalty = 0.8, max_tokens=1000)['choices'][0]['text']
  # for token in llm.generate(prompt):
  #   response += token

  # pattern = r"<p>(.*?)</p>"
  # matches = re.search(pattern, response)

  # if matches:
  #     matched_text = matches.group(1)
  #     return matched_text
  return response

In [16]:
count = 0
template = """
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. Give one unique paraphrase each for Male, Female and Ambiguous.
Change styles based on gender linked language.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. Male : Paraphrased sentence\n2. Female :  Paraphrased sentence\n3. Ambiguous : Paraphrased sentence\n
Document: "{sentence}"
[/INST]
"""
# "Use the sentence given in brackets ({sentence}). Instruction: Paraphrase the sentence while preserving its meaning, in your text reponse give the paraphrased sentence enclosed in <sent></sent>"
# prompt.format(sentence="the sentence")
print("PROMPT - ", template)
for q, c in zip(questions, context):
    llm.reset()
    print("---------------")
    print("Sentence: ", c,"\n")
    prompt = template.format(sentence=c)
    print("Paraphrase: ", get_llm_output(prompt))
    count+=1
    if count==100:
        break

PROMPT -  
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. Give one unique paraphrase each for Male, Female and Ambiguous.
Change styles based on gender linked language.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. Male : Paraphrased sentence
2. Female :  Paraphrased sentence
3. Ambiguous : Paraphrased sentence

Document: "{sentence}"
[/INST]

---------------
Sentence:  Tracy didn't go home that evening and resisted Riley's attacks. 

Paraphrase:  1. Male: "Tracy didn't go home that evening and fought back against Riley's attacks."
2. Female: "Tracy didn't go home that evening and defended herself against Riley's attacks."
3. Ambiguous: "Tracy didn't go home that evening and resisted Riley

In [17]:
def get_llm_output(prompt):
  response = ""
  response = llm(prompt, temperature=0.4, repeat_penalty = 0.2, max_tokens=1000)['choices'][0]['text']
  # for token in llm.generate(prompt):
  #   response += token

  # pattern = r"<p>(.*?)</p>"
  # matches = re.search(pattern, response)

  # if matches:
  #     matched_text = matches.group(1)
  #     return matched_text
  return response

In [ ]:
count = 0
template = """
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. Give one unique paraphrase each for Male, Female and Ambiguous.
Change styles based on gender linked language.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. Male : Paraphrased sentence\n2. Female :  Paraphrased sentence\n3. Ambiguous : Paraphrased sentence\n
Document: "{sentence}"
[/INST]
"""
# "Use the sentence given in brackets ({sentence}). Instruction: Paraphrase the sentence while preserving its meaning, in your text reponse give the paraphrased sentence enclosed in <sent></sent>"
# prompt.format(sentence="the sentence")
print("PROMPT - ", template)
for q, c in zip(questions, context):
    llm.reset()
    print("---------------")
    print("Sentence: ", c,"\n")
    prompt = template.format(sentence=c)
    print("Paraphrase: ", get_llm_output(prompt))
    count+=1
    if count==100:
        break

In [11]:
count = 0
template = """
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. 
Here is what you know about the linguistic style of A and B; statistically A prefer words with more syllables and characters. B prefers describing more psychological states, A prefers conceptual and abstract things. A uses less happier words, B uses more happier words
Give one unique paraphrase each for demographic A, B and ambiguous.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. A : Paraphrased sentence\n2. B :  Paraphrased sentence\n3. Ambiguous : Paraphrased sentence\n
Document: "{sentence}"
[/INST]
"""
# "Use the sentence given in brackets ({sentence}). Instruction: Paraphrase the sentence while preserving its meaning, in your text reponse give the paraphrased sentence enclosed in <sent></sent>"
# prompt.format(sentence="the sentence")
print("PROMPT - ", template)
for q, c in zip(questions, context):
    llm.reset()
    print("---------------")
    print("Sentence: ", c,"\n")
    prompt = template.format(sentence=c)
    print("Paraphrase: ", get_llm_output(prompt))
    count+=1
    if count==100:
        break

PROMPT -  
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. 
Here is what you know about the linguistic style of A and B; statistically A prefer words with more syllables and characters. B prefers describing more psychological states, A prefers conceptual and abstract things. A uses less happier words, B uses more happier words
Give one unique paraphrase each for demographic A, B and ambiguous.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. A : Paraphrased sentence
2. B :  Paraphrased sentence
3. Ambiguous : Paraphrased sentence

Document: "{sentence}"
[/INST]

---------------
Sentence:  Tracy didn't go home that evening and resisted Riley's attacks. 

Paraphrase:  Sure, I'd be happy to help!

In [12]:
count = 0
template = """
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. 
Here is what you know about the traits more likely to be used exhibited A and B; 
A – [More informational and factual, Greater syntactic complexity, Use of determiners, Preference for public forms and actions]
B – [More involved, deeper interaction with the reader, Uses punctuation to show characters' perceptions and feelings, More playful often using puns and word games, Softer, group focused and collaborative]

Give one unique paraphrase each for demographic A, B and Ambiguous.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. A : Paraphrased sentence\n2. B :  Paraphrased sentence\n3. Ambiguous : Paraphrased sentence\n
Document: "{sentence}"
[/INST]
"""
# "Use the sentence given in brackets ({sentence}). Instruction: Paraphrase the sentence while preserving its meaning, in your text reponse give the paraphrased sentence enclosed in <sent></sent>"
# prompt.format(sentence="the sentence")
print("PROMPT - ", template)
for q, c in zip(questions, context):
    llm.reset()
    print("---------------")
    print("Sentence: ", c,"\n")
    prompt = template.format(sentence=c)
    print("Paraphrase: ", get_llm_output(prompt))
    count+=1
    if count==100:
        break

PROMPT -  
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. 
Here is what you know about the traits more likely to be used exhibited A and B; 
A – [More informational and factual, Greater syntactic complexity, Use of determiners, Preference for public forms and actions]
B – [More involved, deeper interaction with the reader, Uses punctuation to show characters' perceptions and feelings, More playful often using puns and word games, Softer, group focused and collaborative]

Give one unique paraphrase each for demographic A, B and Ambiguous.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. A : Paraphrased sentence
2. B :  Paraphrased sentence
3. Ambiguous : Paraphrased sentence

Document: "{sente

In [13]:
count = 0
template = """
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. 
Here is what you know about the traits more likely to be used exhibited A and B; 
A – [direct, forceful, assertive, competitive, task-oriented, less emotional, less polite, less cooperative, more interruptive, more likely to use humor, sarcasm, and insults, more likely to use language to dominate and establish status]
B – [indirect, tentative, expressive, supportive, relationship-oriented, more emotional, more polite, more cooperative, less interruptive, more likely to use tag questions, hedges, qualifiers, compliments, apologies, and personal pronouns]

Give one unique paraphrase each for demographic A, B and Ambiguous.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV, gender as the sentence 
Format: 1. A : Paraphrased sentence\n2. B :  Paraphrased sentence\n3. Ambiguous : Paraphrased sentence\n
Document: "{sentence}"
[/INST]
"""
# "Use the sentence given in brackets ({sentence}). Instruction: Paraphrase the sentence while preserving its meaning, in your text reponse give the paraphrased sentence enclosed in <sent></sent>"
# prompt.format(sentence="the sentence")
print("PROMPT - ", template)
for q, c in zip(questions, context):
    llm.reset()
    print("---------------")
    print("Sentence: ", c,"\n")
    prompt = template.format(sentence=c)
    print("Paraphrase: ", get_llm_output(prompt))
    count+=1
    if count==100:
        break

PROMPT -  
[INST]
<<SYS>>
You are an English language expert.
Tour task is paraphrasing documents based on linguistic style of demographic groups and common knowledge.
<</SYS>>
You will be given a document to paraphrase. 
Here is what you know about the traits more likely to be used exhibited A and B; 
A – [direct, forceful, assertive, competitive, task-oriented, less emotional, less polite, less cooperative, more interruptive, more likely to use humor, sarcasm, and insults, more likely to use language to dominate and establish status]
B – [indirect, tentative, expressive, supportive, relationship-oriented, more emotional, more polite, more cooperative, less interruptive, more likely to use tag questions, hedges, qualifiers, compliments, apologies, and personal pronouns]

Give one unique paraphrase each for demographic A, B and Ambiguous.
Rules: 1. Deviation from the given format is prohibited. 2. You can change the tone and vocabulary depending on the demographic 3. Keep the same POV,